<pre>
<b>
COMP-6721 | 2021-Winter
Project | Part-1

Pravesh Gupta | 40152506
Vikramjeet Singh | 
Manjot Kaur Dherdi | 
</b>
</pre>

# 1. Data Loading

## 1.1. Installing required python modules

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torchvision

In [5]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


### 1.2. Dataset Description


Required Dataset Folder Heirarchy:
<pre>
{dataset_folder}
--images
----mask
-------{mask images}
----no_mask
-------{no mask images}
----not_person
-------{not person images}
</pre>

### 1.3. Defining Pytorch dataset Representation and data transaformers

In [9]:
from torchvision.datasets import VisionDataset
import pandas as pd
import os
from skimage import io as sk_io, transform as sk_transform
import numpy as np
from sklearn.utils import shuffle
from PIL import Image
    
class Rescale(object):
    def __init__(self, output_size, debug=False, export_path=None):
        assert isinstance(output_size, (int))
        self.output_size = output_size

    def __call__(self, img_data):
        img_arr = np.array(img_data)
        scale_factor = float(self.output_size)/img_arr.shape[0]
        img_arr = (sk_transform.rescale(img_arr, (scale_factor, scale_factor, 1))*255).astype(np.uint8)
        
        new_w = img_arr.shape[1]
        
        # Clipping or filling
        if new_w>self.output_size:
            mid = new_w//2
            new_w_start = mid-self.output_size//2
            new_w_end = mid+self.output_size//2
            
            if (new_w_end-new_w_start)<self.output_size:
                new_w_end += (self.output_size-(new_w_end-new_w_start))
            elif (new_w_end-new_w_start)>self.output_size:
                new_w_end -= ((new_w_end-new_w_start)-self.output_size)
            img_arr = img_arr[:, new_w_start:new_w_end]
        elif new_w<self.output_size:
            mid = new_w//2
            new_w_start = self.output_size//2-mid
            new_w_end = new_w_start+new_w
            filled_img_arr = np.zeros((self.output_size, self.output_size, img_arr.shape[2]), dtype=np.uint8)
            filled_img_arr[:, new_w_start:new_w_end] = img_arr[:, :]
            img_arr = filled_img_arr
        return Image.fromarray(img_arr)

### 1.4. Creating dataset and data loaders for train and test

In [19]:
def get_train_test_indices(dataset_targets):
    test_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        test_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    train_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        train_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    targets = np.array(dataset.targets, dtype=np.int32)
    target_indices = np.where(targets!=None)[0]
    np.random.shuffle(target_indices)

    for i in target_indices:
        label = dataset.targets[i]
        if test_indices_map[label]['count']<100:
            test_indices_map[label]['indices'] = np.append(test_indices_map[label]['indices'], i)
            test_indices_map[label]['count']+=1
        elif train_indices_map[label]['count']<350:
            train_indices_map[label]['indices'] = np.append(train_indices_map[label]['indices'], i)
            train_indices_map[label]['count']+=1

    test_indices = np.array([], dtype=np.int32)
    train_indices = np.array([], dtype=np.int32)

    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        label_test_indices = test_indices_map[label]['indices']
        test_indices = np.append(test_indices, label_test_indices)

        label_train_indices = train_indices_map[label]['indices']
        train_indices = np.append(train_indices, label_train_indices)
    return train_indices, test_indices

In [11]:
from torchvision.transforms import ToTensor, Compose, Normalize
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder

data_transform = Compose([
    Rescale(512)
    ,ToTensor()
#     ,Normalize(mean=torch.zeros((3)),std=torch.ones((3)))
])

dataset = ImageFolder(
    root='D:/pravesh/Concordia/2021-Winter/COMP-6721-Intro_To_AI/project/1/dataset/images/'
    ,transform=data_transform
)
print(f'Dataset: size: {len(dataset)}, class labels: {dataset.class_to_idx}')

train_indices, test_indices = get_train_test_indices(dataset.targets)
    
train_dataset = Subset(dataset, train_indices)
print(f'Train dataset: size: {len(train_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
for data, labels in train_dataloader:
    print(f'Train X | y batch shapes : {data.shape, labels.shape}')
    break

test_dataset = Subset(dataset, test_indices)
print(f'Test dataset: size: {len(test_dataset)}')

test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)
for data, labels in test_dataloader:
    print(f'Test X | y batch shapes : {data.shape, labels.shape}')
    break

Dataset: size: 3091, class labels: {'mask': 0, 'no_mask': 1, 'not_person': 2}
Train dataset: size: 1050
Train X | y batch shapes : (torch.Size([10, 3, 512, 512]), torch.Size([10]))
Test dataset: size: 300
Test X | y batch shapes : (torch.Size([10, 3, 512, 512]), torch.Size([10]))


# 2. Data Modelling

In [12]:
from torch.nn import Module, Conv2d, MaxPool2d, Linear, ReLU

class ProjectModel(Module):
    def __init__(self):
        super(ProjectModel, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=2)
        self.pool1 = MaxPool2d(2)
        self.conv2 = Conv2d(in_channels=16, out_channels = 32, kernel_size=3)
        self.linear = Linear(in_features = 32*125*125, out_features=3)
        
    def forward(self, X):
        relu = ReLU(inplace=False)
        X = self.conv1(X)
        X = relu(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = relu(X)
        X = torch.flatten(X, 1)
        X = self.linear(X)
        return X

net = ProjectModel()

### 2.1. Integrity Test

In [34]:
from sklearn import metrics

# Model Unit test
for data, labels in train_dataloader:
    outputs = net(data)
    print(outputs)
    _, y_pred = torch.max(outputs.data, 1)
    print(y_pred)
    prob = torch.nn.functional.softmax(outputs, dim=1)
    print(prob)
    label_pred = torch.argmax(prob, dim=1)
    print(label_pred)
    acc = metrics.accuracy_score(labels, y_pred, normalize=True)
    break
print('Model test passed.')

tensor([[-0.0851,  0.0331, -0.0427],
        [-0.0199,  0.1084, -0.0564],
        [-0.0572,  0.0911, -0.0624],
        [-0.0855,  0.0687, -0.0684],
        [-0.0678,  0.0941, -0.0451],
        [-0.0743,  0.0642, -0.0293],
        [-0.0006,  0.0391, -0.0654],
        [-0.0617,  0.0815, -0.0622],
        [-0.0528,  0.0639, -0.0591],
        [-0.0269,  0.0515, -0.0250]], grad_fn=<AddmmBackward>)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([[0.3156, 0.3552, 0.3292],
        [0.3225, 0.3666, 0.3109],
        [0.3170, 0.3677, 0.3153],
        [0.3141, 0.3665, 0.3195],
        [0.3126, 0.3676, 0.3198],
        [0.3130, 0.3595, 0.3274],
        [0.3358, 0.3494, 0.3148],
        [0.3171, 0.3659, 0.3170],
        [0.3208, 0.3605, 0.3188],
        [0.3243, 0.3508, 0.3249]], grad_fn=<SoftmaxBackward>)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Model test passed.


### 2.2. Training

In [14]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import time

num_epoch = 5
lr = 0.01
momentum = 0.9

loss_evaluater = CrossEntropyLoss()
optimizer = SGD(net.parameters(), lr=lr, momentum=momentum)

t = time.time()
for i in range(num_epoch):
    train_acc_sum = 0.0
    batch_count = 0
    t_epoch = time.time()
    for data, labels in train_dataloader:
        X, y = data, labels
        optimizer.zero_grad()
        output = net(X)
        loss = loss_evaluater(output, y)
        loss.backward()
        optimizer.step()
        _, y_pred = torch.max(output.data, 1)
        acc = metrics.accuracy_score(y, y_pred)
        batch_count+=1
        train_acc_sum+=acc
    epoch_acc = (train_acc_sum/batch_count)
    seconds_passed = time.time()-t_epoch
    print(f'Epoch {i+1}: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {epoch_acc*100}]')
seconds_passed = time.time()-t
print(f'Training time: {seconds_passed//60}m {seconds_passed%60}s')
print('=============================================')

t = time.time()
with torch.no_grad():
    acc_sum = 0.0
    batch_count = 0
    for data, labels in train_dataloader:
        X, y = data, labels
        output = net(X)
        _, y_pred = torch.max(output.data, 1)
        acc = metrics.accuracy_score(y, y_pred)
        acc_sum+=acc
        batch_count+=1
    train_acc = acc_sum/batch_count
    print(f'Training accuracy: {train_acc*100}')
seconds_passed = time.time()-t
print(f'Trainig evaluation time: {seconds_passed//60}m {seconds_passed%60}s')

Epoch 1: [time:15.0m 43.64962816238403s, Accuracy: 34.952380952380935]
Epoch 2: [time:12.0m 5.149068355560303s, Accuracy: 32.38095238095238]
Epoch 3: [time:12.0m 37.05988788604736s, Accuracy: 32.76190476190476]
Epoch 4: [time:11.0m 54.265122175216675s, Accuracy: 32.1904761904762]
Epoch 5: [time:11.0m 51.710593938827515s, Accuracy: 30.95238095238095]
Training time: 64.0m 11.848260879516602s
Training accuracy: 33.33333333333333
Trainig evaluation time: 11.0m 44.382646322250366s


### 2.3. Testing

In [15]:
t = time.time()
acc_sum = 0.0
batch_count = 0
for data, labels in test_dataloader:
    X, y = data, labels
    output = net(X)
    _, y_pred = torch.max(output.data, 1)
    acc = metrics.accuracy_score(y, y_pred)
    acc_sum+=acc
    batch_count+=1
test_acc = acc_sum/batch_count
print(f'Test accuracy: {test_acc*100}')
seconds_passed = time.time()-t
print(f'Test time: {seconds_passed//60}m {seconds_passed%60}s')

Test accuracy: 33.33333333333333
Test time: 3.0m 14.451367855072021s


# 3. Model Evaluation

### 3.1. Accuracy